In [1]:
import os,time,math,pickle,random
import numpy as np,pandas as pd

In [2]:
data_path="/Users/amarlalwani/Desktop/nlp/lm/data/names/"
df=pd.read_csv(data_path+"dataset.csv")
df.dropna(inplace=True)
df["length"]=df["name"].apply(lambda x:len(x))
df=df[(df["nametype"]=="firstname")&(df["length"]<15)]
df

,country,nametype,gender,name,length
0,india,firstname,male,rahul,5
1,india,firstname,male,md,2
2,india,firstname,male,ajay,4
3,india,firstname,male,sanjay,6
4,india,firstname,male,sunil,5
...,...,...,...,...,...
413068,indonesia,firstname,female,ummisideh,9
413069,indonesia,firstname,female,liestyanti,10
413070,indonesia,firstname,female,karwur,6
413071,indonesia,firstname,female,poerwa,6


In [3]:
dataset=[]

In [5]:
n_per_task=5000

In [ ]:
#startswith
rows=df.sample(n_per_task)
for index, row in rows.iterrows():
    prefix_size=min(len(row["name"]),random.choice([1,2,3]))
    prefix=row["name"][:prefix_size]
    datapoint="S"+"="+prefix+"!"+"|"+row["name"]+"|"
    dataset.append(datapoint)

In [ ]:
#endswith
rows=df.sample(n_per_task)
for index, row in rows.iterrows():
    suffix_size=min(len(row["name"]),random.choice([1,2,3]))
    suffix=row["name"][-suffix_size:]
    datapoint="E"+"="+suffix+"!"+"|"+row["name"]+"|"
    dataset.append(datapoint)

In [ ]:
#substring
rows=df.sample(n_per_task)
for index, row in rows.iterrows():
    ss_size=min(len(row["name"]),random.choice([2,3]))
    ss_index=random.choice(np.arange(len(row["name"])-ss_size+1))
    ss=row["name"][ss_index:ss_index+ss_size]
    datapoint="W"+"="+ss+"!"+"|"+row["name"]+"|"
    dataset.append(datapoint)

In [ ]:
#includes the following characters
rows=df.sample(n_per_task)
for index, row in rows.iterrows():
    vocab=list(set(list(row["name"])))
    size=min(len(vocab),random.choice([1,2,3]))
    chars=random.sample(vocab,size)
    datapoint="I"+"="+"".join(chars)+"!"+"|"+row["name"]+"|"
    dataset.append(datapoint)

In [ ]:
#excludes the following characters
rows=df.sample(n_per_task)
for index, row in rows.iterrows():
    universe=list("abcdefghijklmnopqrstuvwxyz")
    vocab=list(set(universe).difference(set(list(row["name"]))))
    size=min(len(vocab),random.choice([1,2,3]))
    chars=random.sample(vocab,size)
    datapoint="X"+"="+"".join(chars)+"!"+"|"+row["name"]+"|"
    dataset.append(datapoint)

In [6]:
#gender
rows=df[df["country"]=="india"].sample(2*n_per_task)
for index, row in rows.iterrows():
    datapoint="G"+"="+row["gender"][0].upper()+"!"+"|"+row["name"]+"|"
    dataset.append(datapoint)

In [ ]:
countries={
    "india":"B",
    "usa":"U",
    "uk":"L",
    "uae":"D",
    "spain":"P",
    "germany":"K",
    "italy":"R",
    "nigeria":"N",
    "morocco":"A",
    "hongkong":"H",
    "singapore":"J",
    "malaysia":"Y",
    "mexico":"O",
    "peru":"Q",
    "netherlands":"T",
    "southafrica":"Z"
}
for country in countries:
    if country not in ["india","peru"]:
        continue
    rows=df[df["country"]==country].sample(n_per_task)
    for index,row in rows.iterrows():
        datapoint="C"+"="+countries[country]+"!"+"|"+row["name"]+"|"
        dataset.append(datapoint)

In [ ]:
#likeness: generate similar name
temp=[]
names=df["name"].tolist()
while True:
    n1,n2=random.sample(names,2)
    ss_size=min(len(n1),len(n2),3)
    n1_ss=set()
    for i in range(len(n1)-ss_size+1):
        n1_ss.add(n1[i:i+ss_size])
    n2_ss=set()
    for i in range(len(n2)-ss_size+1):
        n2_ss.add(n2[i:i+ss_size])
    if len(n1_ss.intersection(n2_ss))>0:
        datapoint="V"+"="+n1+"!"+"|"+n2+"|"
        temp.append(datapoint)
    if len(temp)>=n_per_task:
        dataset+=temp
        break

In [7]:
len(dataset)

10000

In [8]:
train_names=[]
val_names=[]
for name in dataset:
    if random.random()<0.85:
        train_names.append(name)
    else:
        val_names.append(name)

In [9]:
out_dir="instructions"
os.makedirs(out_dir, exist_ok=True)
fp=open(os.path.join(out_dir, 'train.bin'),"w")
fp.write("\n".join(train_names)+"\n")
fp.close()
fp=open(os.path.join(out_dir, 'val.bin'),"w")
fp.write("\n".join(val_names)+"\n")
fp.close()